# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes. 

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [20]:
# INSERT CODE HERE
# install.packages("mlbench") # uncomment if the package is missing
# install.packages("boot")
library(tidyverse)
library(boot)
require(mlbench)


pid2 <- as.data.frame(PimaIndiansDiabetes2) |> # Had to convert to data frame for some reason
    subset(select = -c(insulin)) |> # drop insulin
    drop_na()
head(pid2)

,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset. 

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%. 

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [51]:
# INSERT CODE HERE
glm.fit = glm(diabetes ~ ., data = pid2, family = binomial)    # the `.` includes all other variables
summary(glm.fit)
cv.err = cv.glm(pid2, glm.fit) # not defining K; the default is LOOCV
cv.err$delta


Call:
glm(formula = diabetes ~ ., family = binomial, data = pid2)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -9.554651   0.994217  -9.610  < 2e-16 ***
pregnant     0.122517   0.043743   2.801 0.005097 ** 
glucose      0.035321   0.004244   8.322  < 2e-16 ***
pressure    -0.007695   0.010314  -0.746 0.455602    
triceps      0.006774   0.014759   0.459 0.646242    
mass         0.082678   0.023334   3.543 0.000395 ***
pedigree     1.308708   0.364040   3.595 0.000324 ***
age          0.026375   0.014000   1.884 0.059581 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 676.79  on 531  degrees of freedom
Residual deviance: 466.32  on 524  degrees of freedom
AIC: 482.32

Number of Fisher Scoring iterations: 5


[1] 0.1471790 0.1471734

So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row. 

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [56]:
# Initialize `results` data frame
# INSERT CODE HERE
results <- data.frame(actual = pid2$diabetes)
results$predicted <- NA

#for loop
for (i in 1:nrow(pid2)){ #don't forget to change this to your data set name
    # separate individual observation `i` from the rest of your data
    # INSERT CODE HERE
    test <- pid2[i,]
    train <- pid2[-i,]

    # train your model
    # INSERT CODE HERE
    glm.fit <- glm(diabetes ~ ., data=train, family=binomial)

    # test model on hold out observation
    # INSERT CODE HERE
    predicted_prob = predict(glm.fit, test, type="response") # ensures that the predictions are on the probability scale

    # classify model prediction as "pos" or "neg" and add to `results`
    # INSERT CODE HERE
    predicted_class <- ifelse(predicted_prob > 0.5, "pos", "neg")
    results[i,"predicted"] <- predicted_class
}
head(results)

,actual,predicted
,<fct>,<chr>
1,pos,pos
2,neg,neg
3,neg,neg
4,pos,pos
5,pos,neg
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [60]:
# INSERT CODE HERE
results.err <- mean(results$actual != results$predicted)
print(results.err)

[1] 0.2218045


# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [63]:
?cv.glm

cv.glm                  package:boot                   R Documentation

_C_r_o_s_s-_v_a_l_i_d_a_t_i_o_n _f_o_r _G_e_n_e_r_a_l_i_z_e_d _L_i_n_e_a_r _M_o_d_e_l_s

_D_e_s_c_r_i_p_t_i_o_n:

     This function calculates the estimated K-fold cross-validation
     prediction error for generalized linear models.

_U_s_a_g_e:

     cv.glm(data, glmfit, cost, K)
     
_A_r_g_u_m_e_n_t_s:

    data: A matrix or data frame containing the data.  The rows should
          be cases and the columns correspond to variables, one of
          which is the response.

  glmfit: An object of class '"glm"' containing the results of a
          generalized linear model fitted to 'data'.

    cost: A function of two vector arguments specifying the cost
          function for the cross-validation.  The first argument to
          'cost' should correspond to the observed responses and the
          second argument should correspond to the predict

Here, we see `cost` is defined as: 
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is 

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [66]:
# INSERT CODE HERE
glm.fit = glm(diabetes ~ ., data = pid2, family = binomial)    # the `.` includes all other variables
cost <- function(r, pi = 0) {
    mean(abs(r-pi) > 0.5)
}

cv.err = cv.glm(pid2, glm.fit, cost=cost) # not defining K; the default is LOOCV
cv.err$delta

[1] 0.2218045 0.2221154

How do your results compare to your manual LOOCV above?

> *Write response here*
>
> The first entry of my results (the raw CV error without adjustments) appears to be identical to my manual LOOCV error rate.


# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15. 

In [ ]:
set.seed(1)
#INSERT CODE BELOW

# K = 3
cv.err = cv.glm(pid2, glm.fit, cost=cost, K=3)
cv.err$delta

# K = 5
cv.err = cv.glm(pid2, glm.fit, cost=cost, K=5)
cv.err$delta

# K = 10
cv.err = cv.glm(pid2, glm.fit, cost=cost, K=10)
cv.err$delta

# K = 15
cv.err = cv.glm(pid2, glm.fit, cost=cost, K=15)
cv.err$delta

[1] 0.2105263 0.2168085

[1] 0.2161654 0.2184444

[1] 0.2236842 0.2230942

[1] 0.2274436 0.2281856

#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> *Write response here*
> 
> * The previous LOOCV error is within the range of these four error rates. However, notice that as K increases, the error rate appears to be larger.

If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> *Write response here*
>
> * In this case, we systematically leave on sample out for testing and no random sampling is involved. Therefore, the results will remain the same.


**DUE:** 5pm March 24, 2025

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>
